In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# oversampling packages
from imblearn.combine import SMOTEENN
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import RandomOverSampler

#optimisation
from bayes_opt import BayesianOptimization

# model building
import keras
import tensorflow
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Embedding, SimpleRNN , concatenate, Lambda
from keras.layers import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model, model_selection
import sklearn


In [ ]:
X_train = pd.read_csv('../datasets/word2vec/train_data_imputed_FINAL.csv')
X_test = pd.read_csv('../datasets/word2vec/test_data_imputed_FINAL.csv')
y_train = pd.read_csv('../datasets/word2vec/y_train_FINAL.csv').iloc[:,1:]
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv').iloc[:,1:]

## Oversampling 
- Using Random Sampler

In [ ]:
original_counter = Counter(y_train.loc[:,'fraudulent'])
print("Before count", original_counter)

Before count Counter({0: 11293, 1: 223})


In [ ]:
# oversample word2vec dataset
oversample = RandomOverSampler(sampling_strategy= 1, random_state = 42)
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

In [ ]:
after_counter = Counter(y_train_over.loc[:,'fraudulent'])
print("After count", after_counter)

After count Counter({0: 11293, 1: 11293})


## Hyperparameter Tuning using Bayesian Optimisation

## Hyperparameter tuning 1 

In [ ]:
# Initialising Leaky relu layer to be used in optimisation
LeakyReLU = LeakyReLU(alpha=0.1)

In [ ]:
def rnn_tuner( batch_size, neurons, activation, optimizer, dropout, dropout1, dropout_rate,  dropout_rate1, learning_rate, epochs, layers1, layers2):

    optimizerL = ['SGD', 'Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate)}
    activationL = ['LeakyReLU', 'relu', 'sigmoid', 
                     ]
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def create_model():
      tuned_model = Sequential()
      tuned_model.add(SimpleRNN(units=neurons, input_shape=(263,1))) 

      for i in range(layers1): #varying first set of hidden layers
          if activationL == 'LeakyReLU':
            tuned_model.add(Dense(neurons, activation=activation))
            tuned_model.add(LeakyReLU(alpha=0.1))
          else:
            tuned_model.add(Dense(neurons, activation=activation))
          if dropout > 0.5:
            tuned_model.add(Dropout(dropout_rate, seed=123))

      for i in range(layers2): #varying second set of hidden layers
          if activationL == 'LeakyReLU':
            tuned_model.add(Dense(neurons / 2, activation=activation))
            tuned_model.add(LeakyReLU(alpha=0.1))
          else:
            tuned_model.add(Dense(neurons, activation=activation))
          if dropout1 > 0.5:
            tuned_model.add(Dropout(dropout_rate1, seed=123))

      tuned_model.add(Dense(1, activation='sigmoid')) #output layer

      tuned_model.compile(loss= BinaryCrossentropy(), optimizer = optimizer, metrics=['accuracy','Precision','Recall','AUC'])

      return tuned_model

    es = EarlyStopping(monitor="auc", patience=5)

    tune_rnn = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1) 
    score = cross_val_score(tune_rnn, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring = 'f1').mean() #5 fold cross validation default
    return score

In [ ]:
# Define parameter grid
params_rnn ={
    'neurons': (10, 100),
    'activation':(0,2),
    'optimizer':(0,2),
    'learning_rate':(0.001, 0.005),
    'batch_size':(100, 500),
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'dropout':(0,1),
    'dropout1':(0,1),
    'dropout_rate':(0,0.5),
    'dropout_rate1':(0,0.5)
}

In [ ]:
# Run Bayesian Optimization
rnn_bo = BayesianOptimization(rnn_tuner, params_rnn, random_state=123)
rnn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropout1  | dropou... | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/49
47/47 [==============================] - 6s 74ms/step - loss: 0.3490 - accuracy: 0.8911 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6291
Epoch 2/49
47/47 [==============================] - 3s 74ms/step - loss: 0.2824 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8055
Epoch 3/49
47/47 [==============================] - 3s 71ms/step - loss: 0.2406 - accuracy: 0.8917 - precision: 0.7500 - recall: 0.0056 - auc: 0.8800
Epoch 4/49
47/47 [==============================] - 3s 70ms/step - loss: 0.2003 - accuracy: 0.9072 - precision: 0.6567 - recall: 0.3065 - auc: 0.9259
Epoch 5/49
47/47 [==============================] - 3s 73ms/step - loss: 0.1946 - accuracy: 0.9170 - precision: 0.6616 - recall: 0.4833 - auc: 0.9287
Epoch 6/49
47/47 [==============================] - 3s 70ms/step - loss: 0.1725 - accuracy: 0.9248 - precision: 0.7137 - recall: 0.5148 - auc: 0.9451
Epoch 7/49
47/47 [==============================] - 3s 71ms/step - loss: 0.1408 - ac

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/36
81/81 [==============================] - 9s 88ms/step - loss: 0.3790 - accuracy: 0.8902 - precision: 0.0769 - recall: 9.2593e-04 - auc: 0.5111
Epoch 2/36
81/81 [==============================] - 7s 86ms/step - loss: 0.3643 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5060
Epoch 3/36
81/81 [==============================] - 7s 87ms/step - loss: 0.3462 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5414
Epoch 4/36
81/81 [==============================] - 7s 82ms/step - loss: 0.3464 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5389
Epoch 5/36
81/81 [==============================] - 7s 82ms/step - loss: 0.3450 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5565
Epoch 6/36
81/81 [==============================] - 7s 86ms/step - loss: 0.3350 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6193
Epoch 7/36
81/81 [==============================] - 7s 8

...

Epoch 1/34
28/28 [==============================] - 8s 187ms/step - loss: 0.3984 - accuracy: 0.8632 - precision: 0.1026 - recall: 0.0333 - auc: 0.5082
Epoch 2/34
28/28 [==============================] - 5s 180ms/step - loss: 0.3649 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5070
Epoch 3/34
28/28 [==============================] - 5s 181ms/step - loss: 0.3614 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5125
Epoch 4/34
28/28 [==============================] - 5s 181ms/step - loss: 0.3537 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5141
Epoch 5/34
28/28 [==============================] - 5s 181ms/step - loss: 0.3614 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5104
Epoch 6/34
28/28 [==============================] - 5s 181ms/step - loss: 0.3536 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4993
Epoch 7/34
28/28 [==============================] - 5s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/20
47/47 [==============================] - 10s 177ms/step - loss: 0.3786 - accuracy: 0.8748 - precision: 0.1204 - recall: 0.0241 - auc: 0.5088
Epoch 2/20
47/47 [==============================] - 8s 177ms/step - loss: 0.3452 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4998
Epoch 3/20
47/47 [==============================] - 8s 177ms/step - loss: 0.3438 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5083
Epoch 4/20
47/47 [==============================] - 8s 177ms/step - loss: 0.3447 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5183
Epoch 5/20
47/47 [==============================] - 8s 176ms/step - loss: 0.3438 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5176
Epoch 6/20
47/47 [==============================] - 8s 176ms/step - loss: 0.3438 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5100
Epoch 7/20
47/47 [==============================] - 8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/32
63/63 [==============================] - 8s 90ms/step - loss: 0.6287 - accuracy: 0.7750 - precision: 0.1237 - recall: 0.1759 - auc: 0.5124
Epoch 2/32
63/63 [==============================] - 6s 89ms/step - loss: 0.4639 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5161
Epoch 3/32
63/63 [==============================] - 6s 88ms/step - loss: 0.3703 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5608
Epoch 4/32
63/63 [==============================] - 5s 87ms/step - loss: 0.3441 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6289
Epoch 5/32
63/63 [==============================] - 5s 87ms/step - loss: 0.3374 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6707
Epoch 6/32
63/63 [==============================] - 5s 87ms/step - loss: 0.3333 - accuracy: 0.8913 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7119
Epoch 7/32
63/63 [==============================] - 5s 87ms/

In [ ]:
# Printing optimal parameters derived
params_rnn_ = rnn_bo.max['params']
learning_rate = params_rnn_['learning_rate']
activationL = ['LeakyReLU', 'relu', 'sigmoid'] #elu, tangent
params_rnn_['activation'] = activationL[round(params_rnn_['activation'])]
params_rnn_['batch_size'] = round(params_rnn_['batch_size'])
params_rnn_['epochs'] = round(params_rnn_['epochs'])
params_rnn_['layers1'] = round(params_rnn_['layers1'])
params_rnn_['layers2'] = round(params_rnn_['layers2'])
params_rnn_['neurons'] = round(params_rnn_['neurons'])
optimizerL = ['SGD', 'Adam', 'RMSprop'] 
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                  'RMSprop':RMSprop(learning_rate=learning_rate)}
params_rnn_['optimizer'] = optimizerD[optimizerL[round(params_rnn_['optimizer'])]]
params_rnn_


{'activation': 'relu',
 'batch_size': 367,
 'dropout': 0.02931972289377127,
 'dropout1': 0.6359003593513561,
 'dropout_rate': 0.016098967468962,
 'dropout_rate1': 0.372390327570883,
 'epochs': 34,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.0012670977729000187,
 'neurons': 69,
 'optimizer': <keras.optimizers.optimizer_v2.rmsprop.RMSprop at 0x7f0b41f4d650>}

In [ ]:
tuned_model = Sequential()
tuned_model.add(SimpleRNN(units=params_rnn_['neurons'], input_shape=(263,1))) 

for i in range(params_rnn_['layers1']): #varying first set of hidden layers
    if activationL == 'LeakyReLU':
            tuned_model.add(Dense(params_rnn_['neurons']))
            tuned_model.add(LeakyReLU(alpha=0.1))
    else:
      tuned_model.add(Dense(params_rnn_['neurons'], activation=params_rnn_['activation']))
    tuned_model.add(Dense(params_rnn_['neurons'], activation=params_rnn_['activation']))
    if params_rnn_['dropout'] > 0.5:
      tuned_model.add(Dropout(params_rnn_['dropout_rate'], seed=123))

for i in range(params_rnn_['layers2']): #varying second set of hidden layers
    if activationL == 'LeakyReLU':
            tuned_model.add(Dense(params_rnn_['neurons'] / 2))
            tuned_model.add(LeakyReLU(alpha=0.1))
    else:
      tuned_model.add(Dense(params_rnn_['neurons'] / 2, activation=params_rnn_['activation']))

    if params_rnn_['dropout1'] > 0.5:
      tuned_model.add(Dropout(params_rnn_['dropout_rate1'], seed=123))
tuned_model.add(Dense(1, activation='sigmoid')) #output layer

tuned_model.compile(loss=BinaryCrossentropy(from_logits = False), optimizer = params_rnn_['optimizer'], metrics=['accuracy',metrics.Precision(), metrics.Recall()])
tuned_model.fit(X_train_over, y_train_over, epochs=20)
tuned_model.summary()

Epoch 1/20
389/389 [==============================] - 22s 50ms/step - loss: 0.2961 - accuracy: 0.9077 - precision_14: 0.0526 - recall_14: 8.8574e-04
Epoch 2/20
389/389 [==============================] - 18s 47ms/step - loss: 0.2257 - accuracy: 0.9091 - precision_14: 0.0000e+00 - recall_14: 0.0000e+00
Epoch 3/20
389/389 [==============================] - 19s 49ms/step - loss: 0.1867 - accuracy: 0.9157 - precision_14: 0.6916 - recall_14: 0.1311
Epoch 4/20
389/389 [==============================] - 18s 47ms/step - loss: 0.1519 - accuracy: 0.9377 - precision_14: 0.7194 - recall_14: 0.5155
Epoch 5/20
389/389 [==============================] - 18s 47ms/step - loss: 0.1353 - accuracy: 0.9480 - precision_14: 0.7566 - recall_14: 0.6306
Epoch 6/20
389/389 [==============================] - 19s 48ms/step - loss: 0.1225 - accuracy: 0.9562 - precision_14: 0.7982 - recall_14: 0.6935
Epoch 7/20
389/389 [==============================] - 19s 49ms/step - loss: 0.1050 - accuracy: 0.9637 - precision_14: 

In [ ]:
_,_, precision_train, recall_train = tuned_model.evaluate(X_train_over, y_train_over)

389/389 [==============================] - 8s 19ms/step - loss: 0.0344 - accuracy: 0.9875 - precision_14: 0.9586 - recall_14: 0.9017


In [ ]:
_ ,_ , precision , recall = tuned_model.evaluate(X_test, y_test)

90/90 [==============================] - 3s 30ms/step - loss: 0.1465 - accuracy: 0.9861 - precision_14: 0.6250 - recall_14: 0.7143


In [ ]:
f1_score_train = 2* precision_train * recall_train / (precision_train + recall_train)
f1_score = 2* precision * recall / (precision + recall)

In [ ]:
print(f'F1 score on train set after tuning: {f1_score_train}')
print(f'F1 score on test set after tuning: {f1_score}')

F1 score on train set after tuning: 0.9292560581385739
F1 score on test set after tuning: 0.6666666740841335
